In [3]:
import pandas as pd
import json
import utils
from scipy.spatial import cKDTree
import numpy as np

with open('data/waze.json', 'r') as f:
    data = json.load(f)
    f.close()
alerts = pd.DataFrame(data['alerts'])

In [4]:
alerts = utils.separate_coords(alerts)

alerts.head(3)

,country,city,reportRating,reportByMunicipalityUser,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,endreport,reportDescription,x,y,geometry
0,CI,Antofagasta,1,false,5,10,HAZARD,a123f22e-e5e0-4c6c-8a4e-7434c4fd2110,6.0,17,HAZARD_ON_ROAD_CONSTRUCTION,Av. República de Croacia,1724417637000,1.724425e+12,NaN,-70.412429,-23.676846,POINT (-7838275.742 -2714079.538)
1,CI,Antofagasta,0,false,0,5,JAM,1d8df954-e380-4327-8015-f4fbadf74f2d,6.0,4,JAM_HEAVY_TRAFFIC,Av. Edmundo Pérez Zujovic,1724423878000,1.724425e+12,NaN,-70.392283,-23.617475,POINT (-7836033.099 -2706864.568)
2,CI,Antofagasta,0,false,0,5,JAM,c53a5bc6-f6b6-44b8-a5bd-af69b60a4580,1.0,359,,Pasaje El Volcán,1724425406000,1.724425e+12,NaN,-70.385619,-23.573323,POINT (-7835291.266 -2701501.184)


In [5]:
alerts = alerts[['pubMillis', 'endreport', 'x', 'y', 'geometry']]
alerts.head(3)

,pubMillis,endreport,x,y,geometry
0,1724417637000,1.724425e+12,-70.412429,-23.676846,POINT (-7838275.742 -2714079.538)
1,1724423878000,1.724425e+12,-70.392283,-23.617475,POINT (-7836033.099 -2706864.568)
2,1724425406000,1.724425e+12,-70.385619,-23.573323,POINT (-7835291.266 -2701501.184)


In [8]:
# Verifica si hay alertas cercanas en el mismo periodo de tiempo, con el fin de evaluar si hay reportes duplicados

rad = 300
coords = np.vstack([alerts.geometry.x, alerts.geometry.y]).T
tree = cKDTree(coords)
duplicates = tree.query_ball_point(coords, r=rad)

duplicates
### TODO: Continuar revisando si hay reportes duplicados cruzando con tiempo

array([list([0, 20, 83, 115, 187, 399, 445, 560, 606, 608, 1169, 1182, 1729, 1806, 1808, 2186, 2187, 2379, 2382, 2533, 2596, 3253, 3493, 3715, 3716, 3720, 3721, 3767, 4757, 4758, 4914, 5193, 5832, 5880, 7114, 7277, 7375, 8211, 8467, 8471, 8666, 8680, 8748, 8897, 8911, 9054, 9056, 9235, 9307, 9338, 9584, 9659, 9674, 9944, 10705, 10932, 10942, 10944, 10951]),
       list([1, 21, 29, 104, 169, 299, 389, 390, 391, 492, 529, 555, 598, 638, 664, 728, 809, 964, 1031, 1072, 1109, 1111, 1112, 1146, 1231, 1232, 1257, 1319, 1469, 1504, 1517, 1526, 1559, 1578, 1636, 1773, 1959, 1981, 2241, 2250, 2286, 2322, 2326, 2328, 2401, 2448, 2458, 2461, 2523, 2571, 2695, 2753, 2755, 2761, 2789, 2851, 2866, 2883, 3014, 3020, 3092, 3107, 3195, 3198, 3256, 3296, 3299, 3474, 3597, 3599, 3959, 3964, 3989, 4046, 4072, 4079, 4138, 4256, 4266, 4271, 4273, 4275, 4281, 4282, 4292, 4318, 4361, 4456, 4466, 4534, 4620, 4661, 4805, 4872, 4954, 4957, 5199, 5242, 5253, 5267, 5382, 5410, 5427, 5490, 5524, 5527, 5619, 5908, 5

In [16]:
millisday = 24 * 60 * 60 * 1000

In [19]:
duplicate_reports = []

for list_of_duplicates in duplicates:
    for i in range(len(list_of_duplicates)):
        for j in range(i + 1, len(list_of_duplicates)):
            if np.abs(alerts.iloc[list_of_duplicates[i]].pubMillis - alerts.iloc[list_of_duplicates[j]].pubMillis) < millisday / 2:
                duplicate_reports.append(j)

KeyboardInterrupt: 

In [7]:
print(f'Hay {len(duplicate_reports)} alertas que se superponen')
print('Por lo que se puede considerar como independientes unas de otras')

AttributeError: 'GeoDataFrame' object has no attribute 'reports'